In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.datasets import TranslationDataset, Multi30k, IWSLT, WMT14
from torchtext.data import Field, BucketIterator, Iterator
import spacy
import numpy as np
import random
import math
import time
from collections import defaultdict
from matplotlib import pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import itertools

In [0]:
%%capture
!python3 -m spacy download en
!python3 -m spacy download de

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
sos_token='<sos>'
eos_token='<eos>'
pad_token='<pad>'
unk_token='<unk>'

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [0]:
TRG = Field(init_token=sos_token, 
            eos_token=eos_token,
            lower=True, 
            tokenize=tokenize_en,
            pad_token=pad_token, 
            unk_token=unk_token)

SRC = Field(init_token=sos_token, 
            eos_token=eos_token,
            lower=True, 
            tokenize=tokenize_de,
            pad_token=pad_token, 
            unk_token=unk_token)

In [0]:
%%capture
train_data, _, _ = IWSLT.splits(exts = ('.de', '.en'), 
                        fields = (SRC, TRG),
                        filter_pred=lambda x: len(x.__dict__['src']) <= 50)

test_10, valid_10, valid_12 = IWSLT.splits(exts = ('.de', '.en'), 
                                fields = (SRC, TRG),
                                train='IWSLT16.TED.tst2010',
                                test='IWSLT16.TED.dev2010',
                                validation='IWSLT16.TEDX.dev2012',
                                filter_pred=lambda x: len(x.__dict__['src']) <= 60)

test_11, test_12, test_13 = IWSLT.splits(exts = ('.de', '.en'), 
                        fields = (SRC, TRG),
                        train='IWSLT16.TED.tst2011',
                        validation='IWSLT16.TED.tst2012',
                        test='IWSLT16.TED.tst2013',
                        filter_pred=lambda x: len(x.__dict__['src']) <= 60)

test_13x, test_14x, test_14 = IWSLT.splits(exts = ('.de', '.en'), 
                        fields = (SRC, TRG),
                        train='IWSLT16.TEDX.tst2013',
                        validation='IWSLT16.TEDX.tst2014',
                        test='IWSLT16.TED.tst2014',
                        filter_pred=lambda x: len(x.__dict__['src']) <= 60)

valid_data = [valid_10, valid_12]
test_data = [test_10, test_11, test_12, test_13, test_13x, test_14x, test_14]

In [0]:
SRC.build_vocab(train_data, max_size=10000)
TRG.build_vocab(train_data, max_size=10000)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
class EncoderAttnBi(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden[1]

class AttentionBi(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear(2 * enc_hid_dim + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
        
        return F.softmax(attention, dim=1)

class DecoderAttnBi(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        return prediction, hidden.squeeze(0)

class SearchBi(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

        
    def translate(self, sentence, max_len=50, beam_width=3):
        self.eval()
        src = SRC.process([sentence]).to(device)
        encoder_out, hidden = self.encoder(src)
        beams = [(0.0, [TRG.vocab.stoi[TRG.init_token]], hidden)]
        done = False
        log_smax = nn.LogSoftmax(dim=0).to(device)

        while not done:
            for i in range(len(beams)):
                p, sent, hidden = beams.pop(0)
                if len(sent) >= max_len or sent[-1] == TRG.vocab.stoi[TRG.eos_token]:
                    beams += [(p, sent, hidden)]
                    continue
                trg = torch.ones(1, dtype=torch.int64).to(device) * sent[-1]
                pred, hidden = self.decoder(trg, hidden, encoder_out)
                ll = log_smax(pred[0])
                top_ll, top_t = torch.topk(ll, k=beam_width)
                beams += [(p + ll, sent + [t], hidden) for ll, t in zip(top_ll, top_t)]
            beams = sorted(beams, reverse=True)[:beam_width]
            done = all([b[1][-1] == TRG.vocab.stoi[TRG.eos_token] or len(b[1]) >= max_len for b in beams])

        trgs = beams.pop(0)[1]
        return [TRG.vocab.itos[i] for i in trgs]

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
HID_DIM = [200, 400, 600, 800, 1000]
MAX_LENGTH = 50

att = [AttentionBi(h, h) for h in HID_DIM]
enc = [EncoderAttnBi(INPUT_DIM, ENC_EMB_DIM, h, h)for h in HID_DIM]
dec = [DecoderAttnBi(OUTPUT_DIM, DEC_EMB_DIM, h, h, a) for h, a in zip(HID_DIM, att)]
search_bi = [SearchBi(e, d, device).to(device) for e, d in zip(enc, dec)]

In [15]:
res = []
for i in range(len(HID_DIM)):
    res.append([])
    for epoch in range(6):
        search_bi[i].load_state_dict(torch.load(f'/content/drive/My Drive/ml-mini-project/hidden/attention-model-{HID_DIM[i]}-{epoch}.pt'))
        res[i].append(evaluate_bleu(search_bi[i], valid_data))

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........


In [0]:
def evaluate_bleu(model, datasets, ignore_unk=False):
    total = 0.0
    count = 0.0
    scores = np.zeros(sum([len(d) for d in datasets]))
    for i, example in enumerate(itertools.chain(*datasets)):
        sent = SRC.process([example.src])
        if ignore_unk and SRC.vocab.stoi[SRC.unk_token] in sent:
            continue
        pred = model.translate(example.src)[1:-2]
        score = sentence_bleu([example.trg], pred, smoothing_function=SmoothingFunction().method1)
        total += score
        count += 1
        scores[i] = score
        if i % 1000 == 999: print('.', end='')
    print('')
    return scores
    
def bleu_summary(model, datasets, ignore_unk=False):
    overall_examples = sum([len(d) for d in datasets])
    lengths = np.zeros(overall_examples, dtype=np.int32)
    scores = np.zeros(overall_examples)

    for i, example in enumerate(itertools.chain(*datasets)):
        sent = SRC.process([example.src])
        if ignore_unk and SRC.vocab.stoi[SRC.unk_token] in sent:
            continue
        pred = model.translate(example.src)
        scores[i] = sentence_bleu([example.trg], pred, smoothing_function=SmoothingFunction().method1)
        lengths[i] = len(example.src)
        if i % 900 == 899: print('.', end='')
      
    scores = scores[lengths != 0.0]    
    lengths = lengths[lengths != 0.0]

    print('')
    means, up, lo = {}, {}, {}
    uniq_lengths, counts = np.unique(lengths, return_counts=True)
    for l, c in zip(uniq_lengths, counts):
        s = scores[lengths == l]
        means[l] = s.mean()
        up[l] = means[l] + (np.sqrt(s.var() / c) * 1.65)
        lo[l] = means[l] - (np.sqrt(s.var() / c) * 1.65)

    uniq_lengths = [l for l in sorted(uniq_lengths)]
    return uniq_lengths, [means[l] for l in uniq_lengths], [up[l] for l in uniq_lengths], [lo[l] for l in uniq_lengths]

In [20]:
search_bi[0].load_state_dict(torch.load(f'/content/drive/My Drive/ml-mini-project/hidden/attention-model-200-5.pt'))
search_bi[1].load_state_dict(torch.load(f'/content/drive/My Drive/ml-mini-project/hidden/attention-model-400-5.pt'))
search_bi[2].load_state_dict(torch.load(f'/content/drive/My Drive/ml-mini-project/hidden/attention-model-600-5.pt'))
search_bi[3].load_state_dict(torch.load(f'/content/drive/My Drive/ml-mini-project/hidden/attention-model-800-5.pt'))
search_bi[4].load_state_dict(torch.load(f'/content/drive/My Drive/ml-mini-project/hidden/attention-model-1000-5.pt'))

<All keys matched successfully>

In [27]:
res = []
for s in search_bi:
    res.append(evaluate_bleu(s, test_data))

.........
.........
.........
.........
.........


In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [0]:
# means = [d.mean() for d in res]
n = 9663

In [0]:
# res = [0.16942321591401932, 0.17180345054113969, 0.1702285792543723, 0.1708189162399337, 0.1655442693664769]
counts = [16280004, 24986404, 35132804, 46719204, 59745604]
mean = [d.mean() for d in res]
var_ = [d.var() for d in res]
diff = [(1.65 * np.sqrt(v / n)) for m, v in zip(mean, var_)]

In [0]:
fig, ax = plt.subplots(figsize=(10,8))
ax.set_axisbelow(True)
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.tick_params(which='both', top='off', left='off', right='off', bottom='off')

labels = ['200', '400', '600', '800', '1000']
rect1 = ax.errorbar(labels, [r*100 for r in res], yerr=diff, color='grey', alpha=0.7)
ax.set_xlabel('Hidden Units')
ax.set_ylabel('Average BLEU Score')
ax.set_title('BLEU Score Performance by Number of Hidden Units')
ax.set_ybound(16.4, 17.4)
# alt = ax.twinx()
# alt.set_ylabel('Number of Trainable Parameters')
# alt.bar(labels, counts, color='grey')

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{round(height, 2)}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rect1)

plt.show()